In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
from shell.learners.fl_utils import *
import copy

from shell.fleet.network import *
logging.basicConfig(level=logging.INFO)

In [9]:
experiment_folder = "."
# experiment_folder = "rerun_opt_heuristic_experiment_results"
experiment_name = "more_fl_fix_root_agent_results/fedprox_mu_0.1_comm_freq_5"
dataset = "mnist"
# algo = "modular"
algo = "monolithic"

use_contrastive = False
num_trains_per_class = 64 if dataset != "cifar100" else 256
seed = 0
parallel = False
task_id = 4

In [10]:
modify_cfg=None

In [11]:
save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)

fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
fleet.change_save_dir('debug_more_fl_results')

{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True}, 'sharing_strategy': {'name': 'fedprox', 'num_coms_per_round': 1, 'comm_freq': 5, 'mu': 0.1, 'when_reoptimize_structure': 'never', 'sync_base': True, 'pre_or_post_comm': 'pre'}, 'seed': 0, 'algo': 'monolithic', 'job_name': 'mnist_monolithic_numtrain_64', 'num_agents': 8, 'root_save_dir': 'more_fl_fix_root_agent_results/fedprox_mu_0.1_comm_freq_5', 'parallel': True, 'num_init_tasks': 4, 'overwrite': False, 'topology': 'fully_connected', 'edge_drop_prob': 0.0, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memo

INFO:root:Class sequence: [5 0 4 3 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1954, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (2115, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1901, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:Class sequence: [4 7 5 7 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1920, 1, 28, 28)
INFO:root:task 2 :(128, 

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 4, 'use_contrastive': False}
<class 'shell.learners.er_nocomponents.NoComponentsER'>


INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (2042, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (2145, 1, 28, 28)
INFO:root:Agent: node_id: 69420, seed: 69420000
INFO:root:Agent: node_id: 0, seed: 0
INFO:root:Agent: node_id: 1, seed: 1000
INFO:root:Agent: node_id: 2, seed: 2000
INFO:root:Agent: node_id: 3, seed: 3000
INFO:root:Agent: node_id: 4, seed: 4000
INFO:root:Agent: node_id: 5, seed: 5000
INFO:root:Agent: node_id: 6, seed: 6000
INFO:root:Agent: node_id: 7, seed: 7000
INFO:root:Created fleet with 8 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized


Setting seed 69420000
Setting seed 0
Setting seed 1000
Setting seed 2000
Setting seed 3000
Setting seed 4000
Setting seed 5000
Setting seed 6000
Setting seed 7000


In [12]:
fleet.train_and_comm(task_id=task_id+1)

INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4
INFO:root:AGGREGATING MODELS...no_components 4


from 0 to 5 final False


INFO:root:epochs: 0, training task: 5
INFO:root:	task: 0	loss: 0.09933410	acc: 0.973
INFO:root:	task: 1	loss: 0.30431303	acc: 0.938
INFO:root:	task: 2	loss: 0.36044463	acc: 0.902
INFO:root:	task: 3	loss: 0.21486674	acc: 0.956
INFO:root:	task: 4	loss: 0.48018463	acc: 0.706
INFO:root:	task: 5	loss: 0.65858134	acc: 0.537
INFO:root:	task: avg	loss: 0.35295408	acc: 0.835
INFO:root:epochs: 1, training task: 5
INFO:root:	task: 0	loss: 0.07095879	acc: 0.980
INFO:root:	task: 1	loss: 0.30101478	acc: 0.914
INFO:root:	task: 2	loss: 0.37637609	acc: 0.905
INFO:root:	task: 3	loss: 0.20967555	acc: 0.957
INFO:root:	task: 4	loss: 0.37594168	acc: 0.847
INFO:root:	task: 5	loss: 0.32837375	acc: 0.959
INFO:root:	task: avg	loss: 0.27705677	acc: 0.927
INFO:root:epochs: 0, training task: 5
INFO:root:	task: 0	loss: 0.09826708	acc: 0.972
INFO:root:	task: 1	loss: 0.30048993	acc: 0.933
INFO:root:	task: 2	loss: 0.39017496	acc: 0.902
INFO:root:	task: 3	loss: 0.23385756	acc: 0.956
INFO:root:	task: 4	loss: 0.27666355	

KeyboardInterrupt: 

In [ ]:
# fleet.communicate(task_id=task_id, end_epoch=None, comm_freq=None, num_epochs=None,
#                       final=False)

In [ ]:
# agent = fleet.agents[0]
# # agent.incoming_models[1].keys()

In [ ]:
# agent.net.structure[0]

In [ ]:
# model_L2(agent.net, copy.deepcopy(agent.net), verbose=True, excluded_params=agent.excluded_params)